In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "" #"1"

# set directory
os.chdir("/nas/ucb/oliveradk/diverse-gen/")

In [2]:
# for now incomplete spurious correlation: 
#   waterbirds

# other notebooks:
# multi-class classification: 
#   multi-nli-cc

# known group labels
# waterbirds (normal)
# mulit-nli cc 


In [3]:
from tqdm import tqdm
from pathlib import Path
from datetime import datetime
from itertools import product
import json
import copy

import numpy as np
import nevergrad as ng 

from losses.loss_types import LossType
from utils.exp_utils import get_executor, get_executor_local, run_experiments

In [4]:
SCRIPT_NAME = "spur_corr_exp.py"
EXP_DIR = "output/real_data_group_labels_aux_weight_sweep"
n_trials = 32

In [5]:
method_configs = {
    "DivDis": {"loss_type": LossType.DIVDIS},
    "TopK 0.1": {"loss_type": LossType.TOPK, "mix_rate_lower_bound": 0.1},
    "TopK 0.5": {"loss_type": LossType.TOPK, "mix_rate_lower_bound": 0.5},
    "DBAT": {"loss_type": LossType.DBAT, "shared_backbone": False, "freeze_heads": True, "binary": True, "batch_size": 16, "target_batch_size": 32},
}

dataset_configs = {
    "waterbirds": {"dataset": "waterbirds", "model": "Resnet50", "epochs": 5, "source_cc": False, "use_group_labels": True},
}

method_ranges = {
    "DivDis": {"aux_weight": ng.p.Log(lower=1e-0, upper=1e1)},
    "TopK 0.1": {"aux_weight": ng.p.Log(lower=1e-0, upper=1e1)},
    "TopK 0.5": {"aux_weight": ng.p.Log(lower=1e-0, upper=1e1)},
    "DBAT": {"aux_weight": ng.p.Log(lower=1e-1, upper=1e1)},
}

configs = {
    (ds_name, method_name): {**ds_config, **method_config} 
    for (ds_name, ds_config), (method_name, method_config) in product(dataset_configs.items(), method_configs.items())
}


def sample_hparams(hparam_ranges):
    instrum = ng.p.Instrumentation(**hparam_ranges)
    sampled = instrum.sample()
    return sampled.kwargs


def get_conf_exp_dir(ds_name, method_name, idx):
    return Path(EXP_DIR, f"{ds_name}_{method_name}/{idx}")

sampled_configs = []
for (ds_name, method_name), conf in configs.items():
    for i in range(n_trials):
        sample_config = copy.deepcopy(conf)
        seed = np.random.randint(0, 1000000)
        sample_config["seed"] = seed
        sampled_hparams = sample_hparams(method_ranges[method_name])
        sample_config.update(sampled_hparams)
        sample_config["exp_dir"] = get_conf_exp_dir(ds_name, method_name, i)
        sample_config["plot_activations"] = False
        sampled_configs.append(sample_config)

In [9]:
# executor = get_executor_local(EXP_DIR + "test")
# jobs = run_experiments(executor, sampled_configs[:1], SCRIPT_NAME)

# Run Experiments

In [6]:
executor = get_executor(EXP_DIR, mem_gb=16)
jobs = run_experiments(executor, sampled_configs, SCRIPT_NAME)

# Plot Results

In [6]:
import yaml
import pandas as pd
all_results = []
not_found_configs = []
for (ds_name, method_name), config in configs.items():
    parent_dir = Path(EXP_DIR, f"{ds_name}_{method_name}")
    for exp_dir in parent_dir.iterdir():
        if not exp_dir.is_dir():
            continue
        metrics_path = Path(exp_dir, "metrics.json")
        config_path = Path(exp_dir, "config.yaml")
        if not metrics_path.exists():
            # print(f"ds_name: {ds_name}, method_name: {method_name}, i: {i}")
            not_found_configs.append((ds_name, method_name, i))
            continue
        with open(metrics_path, "r") as f:
            metrics = json.load(f)
        with open(config_path, "r") as f:
            config = yaml.safe_load(f)
        min_val_loss = min(metrics["val_loss"])
        all_results.append({
            "dataset": ds_name,
            "method_name": method_name,
            "aux_weight": config["aux_weight"],
            "lr": config["lr"],
            "loss": min_val_loss,
        })

df = pd.DataFrame(all_results, columns=["dataset", "method_name", "aux_weight", "loss"])

In [7]:
df

,dataset,method_name,aux_weight,loss
0,waterbirds,DivDis,1.791837,0.188491
1,waterbirds,DivDis,3.914801,0.205598
2,waterbirds,DivDis,6.631930,0.188535
3,waterbirds,DivDis,1.098419,0.201328
4,waterbirds,DivDis,3.568436,0.196053
...,...,...,...,...
123,waterbirds,DBAT,0.119611,1.485808
124,waterbirds,DBAT,0.142990,1.610926
125,waterbirds,DBAT,0.526466,1.433473
126,waterbirds,DBAT,1.090742,1.493447


In [14]:
import pandas as pd
import plotly.graph_objects as go

def plot_aux_weight_vs_loss(df, dataset=None, method_name=None, use_log_scale=True):
    # Filter data based on conditions
    plot_df = df.copy()
    if dataset:
        plot_df = plot_df[plot_df['dataset'] == dataset]
    if method_name:
        plot_df = plot_df[plot_df['method_name'] == method_name]
    
    # Create figure
    fig = go.Figure()
    
    # Group by conditions that weren't filtered
    groupby_cols = []
    if not dataset:
        groupby_cols.append('dataset')
    if not method_name:
        groupby_cols.append('method_name')
    
    if groupby_cols:
        # Plot separate lines for each group
        for name, group in plot_df.groupby(groupby_cols):
            name_str = '_'.join([str(n) for n in name]) if isinstance(name, tuple) else str(name)
            # Sort by aux_weight before plotting
            group = group.sort_values('aux_weight')
            fig.add_trace(go.Scatter(
                x=group['aux_weight'],
                y=group['loss'],
                name=name_str,
                mode='markers+lines'
            ))
    else:
        # Single line plot
        # Sort by aux_weight before plotting
        print(len(plot_df))
        plot_df = plot_df.sort_values('aux_weight')
        print(len(plot_df))
        fig.add_trace(go.Scatter(
            x=plot_df['aux_weight'],
            y=plot_df['loss'],
            mode='markers+lines'
        ))
    
    # Update layout
    title = 'Aux Weight vs Loss'
    if dataset:
        title += f' for {dataset}'
    if method_name:
        title += f' with {method_name}'
    
    fig.update_layout(
        title=title,
        xaxis_title='Aux Weight',
        yaxis_title='Loss',
        width=800,
        height=500
    )
    
    if use_log_scale:
        fig.update_xaxes(type='log')
        fig.update_yaxes(type='log')
    
    return fig

method_name = "DBAT"
fig = plot_aux_weight_vs_loss(df, method_name=method_name)
fig.show()

In [ ]:
# rouhgly seems like 2.5 is good for all of them